# -------------------------------------------------------------------------------------
# Постановка задачи: нам предлагается файл с данными о покупках различных паков в игре W.. 
# Необходимо изучить ассоциативные правила, которые мы можем построить на основании предложенных данных.
# Наибольший интерес вызывают акционные паки.
# -------------------------------------------------------------------------------------

# *LEVEL 1:*

## Для начала посмотрим на [диаграмму Ганта](https://public.tableau.com/profile/roman2610#!/vizhome/Book_207/Dashboard2).

Она позволяет визуально представить в какие дни были совершены покупки и на основании этого выделить паттерн акционных паков.

In [1]:
from datetime import date
from datetime import datetime

import csv
import sys, errno

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser

Модуль **mod_apriori_1** содержит функции для 1-3 уровней исследования.

In [2]:
import mod_apriori_1 as m

Посмотрим сколько всего различных паков наблюдается по нашим данным:

In [3]:
pcks=m.all_packs()

Всего паков:  214


Акционный пак будем выделять таким образом:
- разобьем все покупки на периоды (2 покупки относятся к одному периоду, если они были совершены с разницей в неболее, чем 2 дня)
- если каждый период длится не более 2х недель (примерное значение, полученное путем рассмотрения диаграммы Ганта), то пак считается акционным

Выделим все акционные паки:

In [4]:
now = datetime.now()
action_pack=m.find_action_packs_with_periods("all", pcks) # любая строка, кроме fb, vk, mm, ok будет искать все паки
now1 = datetime.now()
print (now1-now)

0:00:24.904127


Априори будет применяться к каждому акционному периоду.

Для этого выделим всевозможные периоды (некоторые периоды входят друг в друга, поэтому их выбираем, как один период):

In [5]:
times_s, times_e = m.periods_without_intersections(action_pack)
# times_s - начальные времена
# times_e - конечные времена  (пример периода: times_s[0] - times_e[0])
#print times_s,"\n\n", times_e

Создаем словарь ID : паки - времена

In [6]:
id_pack_period=m.dict_id_pack_periods()

Теперь запускаем Априори. На выходе мы получим файл с правилами (порядок покупок в рамках периода учтен).

In [7]:
tabl_dict={} # словарь с необходимыми данными по правилам
rules_for_apriori={} # словарь с транзакциями правил (для составления стратегий)
rules_definition={} # словарь инициализирующий правила из предыдущего словаря
tabl_dict, rules_for_apriori, rules_definition=m.write_res_to_file_name("res_l1_check_seq_1.txt", 0.02, 0.5,tabl_dict, rules_for_apriori, rules_definition, id_pack_period, times_s, times_e, action_pack)

0:00:33.254648


Создаем файл (правило-информация) для `Tableau`:

In [8]:
with open('good_thing_to_do_1.csv', 'w') as csvfile_1:
    fieldnames=["ID","Rule","Confidence","Lift","Support","Social_network","Date","Number_of_transactions_in_period","Consist_action_pack"]
    writer = csv.DictWriter(csvfile_1, fieldnames=fieldnames,delimiter='|')
    writer.writeheader()
    for key,value in tabl_dict.items():
        #print value[4],
        writer.writerow({fieldnames[0]: key, fieldnames[1]: value[0], fieldnames[2]: value[1], fieldnames[3]: value[2],fieldnames[4]: value[3],fieldnames[5]: value[4],fieldnames[6]: value[5],fieldnames[7]: value[6],fieldnames[8]: value[7]})

## *Результаты по 1-му уровню*:

*Основные результаты представлены в виде дэшбордов.*

1. [Правило - информация (1)](https://public.tableau.com/profile/publish/Rule_Inf_about_it/Dashboard3#!/publish-confirm)     
Данный дэшборд показывает получившиеся правила и информацию о них (наличие акционного пака в правиле, количество транзакций в периоде, support, confidence, lift). **Рассматриваються все периоды всех паков (взможны повторяющиеся результаты при совпадении периодов), порядок не учитывается.**

2. [Правило - информация (2)](https://public.tableau.com/profile/publish/Rule-Infconsideringsequence/Dashboard1#!/publish-confirm)     
Данный дэшборд показывает получившиеся правила и информацию о них. Рассматриваються все периоды всех паков (возможны повторяющиеся результаты при совпадении периодов), порядок  **учитывается**.

3. [Правило - информация (3)](https://public.tableau.com/profile/publish/Rule-Informationconsideringsequenceandperiods/Dashboard1#!/publish-confirm)     
Данный дэшборд показывает получившиеся правила и информацию о них. Рассматриваются **периоды, выделенные алгоритмом**, порядок  **учитывается.**

'*Основные моменты: чем больше транзакций, тем лучше (на наших данных, при кол-ве транзакций >100, в большинстве это правила внутри соц. сети - одноклассники), нужно исключить лифт = 1 (иначе данные независимы).*

## *Выводы по 1-му уровню:*

1. Если выделить только акционные правила, то можно заметить, что во всех периодах наблюдается слишком мало транзакций. Такой результат можно интерпретировать **либо как то, что в большинстве акции не влияют на результаты покупок **(если, к примеру, акции должны приводить к покупкам данных паков пользователей с различных соц.сетей), **либо, как то, что необходимо рассматривать правила по некоторым дополнительным признакам (как, к примеру, внутри одной соц. сети)**, т.к. иначе акционные паки не проходят порог минимального саппорта, который в нашем случае итак слишком маленький.

2. Учитывая *основные моменты*' **(numb >= 100, conf >= 0.7, lift > 1, sup > 0.1)**, посмотрим какие правила нам удалось получить (Правило-информация (1)). 
В основном получилось много одинаковых правил с различными показателями (conf, sup. ...). К тому же, среди них нет ни одного акционного пака.

[Правила](https://public.tableau.com/shared/G75C86N92?:display_count=yes):

    - ok_offer_new4  =>  ok_offer_new3
    - ok_offer_new3  =>  ok_crystal_upgrade_2
    - (i, j)  =>  k,    
    i,j,k in {ok_offer_new1,ok_offer_new2,ok_offer_new3}

Таким образом, можно заметить, что самые удачные правила, все до единого принадлежат соц. сети **OK**. Это можно объяснить скорее нехваткой данных, нежели какими-то более смелыми выводоми. Однако более приемлемым результатом можно выделить то, что из стандартных паков в **OK**, только вышеприведенные можно считать зависимыми друг от друга (**акционных паков нет!**).



# *LEVEL 2:*

Модуль **mod_apriori_lvl2** содержит функции для 2 уровня исследования.

In [9]:
import mod_apriori_lvl2 as ml2

## Промежуточный результат:

При добавлении страны в транзакцию, работая со всеми соц. сетями получались тривиалные результаты, по типу: US => fb, RU => vk/ok. Поэтому мы проводили исследования в рамках одной соц. сети (провели по каждой).

In [34]:
social_ntwrk="vk"
now = datetime.now()
action_pack_2=m.find_action_packs_with_periods(social_ntwrk, pcks) # любая строка, кроме fb, vk, mm, ok будет искать все паки
now1 = datetime.now()
print (now1-now)

0:00:06.392363


Создаем словарь id: социальные данные:

In [35]:
id_soc_data=ml2.id_soc_data_inf()

Error has been found
0:00:07.079810
0:00:02.437796


In [36]:
id_pack_period_2=ml2.id_pack_period_lvl2(social_ntwrk) # на вход подаем какая social network исследуется

0:00:00.331613


Запускаем Априори, где в транзакциях, помимо паков, присутствуют страны. (**порядок не учитывается!**; **промежутки не выделяются!** (могут быть пересечения))

In [37]:
tabl_dct={}
tabl_dct=ml2.write_res_to_file_name_lvl2("res_l1_check_2.txt", 0.1, 0.65,tabl_dct, action_pack_2, id_soc_data, id_pack_period_2)

0:00:06.161527


Создаем файл (правило-информация) для `Tableau`:

In [38]:
with open('good_thing_to_do_with_country_vk.csv', 'w') as csvfile_1:
    fieldnames=["ID","Rule","Confidence","Lift","Support","Social_network","Date","Number_of_transactions_in_period","Consist_action_pack","Consists_country"]
    writer = csv.DictWriter(csvfile_1, fieldnames=fieldnames,delimiter='|')
    writer.writeheader()
    count =1
    for key,value in tabl_dct.items():
        l_s=""
        r_s=""
        acs1=False
        acs2=False
        for i in value[0]:
            if i!="=" and acs1==False:
                l_s=l_s+i
            else:
                acs1=True
                if i!="="and i!=">" and i!=" " and acs2==False:
                    r_s=r_s+i
        consists_country="no"
        l=l_s.strip(" ").strip("(").strip(")").strip(",").split(",")
        for i in range(0,len(l)):
            l[i]=l[i].strip("'")
            if l[i][0:2]!=social_ntwrk:
                if count!=4:
                    count+=1
                    #print l,"===",value[0]
                consists_country="yes"
                        
        r=r_s.strip(" ").strip("(").strip(")").strip(",").split(",")
        for i in range(0,len(r)):
            r[i]=r[i].strip("'")
            if r[i][0:2]!=social_ntwrk:
                consists_country="yes"
                if count!=4:
                    count+=1
                    #print r,"===",value[0]
        
        writer.writerow({fieldnames[0]: key, fieldnames[1]: value[0], fieldnames[2]: value[1], fieldnames[3]: value[2],fieldnames[4]: value[3],fieldnames[5]: value[4],fieldnames[6]: value[5],fieldnames[7]: value[6],fieldnames[8]: value[7],fieldnames[9]: consists_country})

## *Результаты по 2-му уровню*:

-  [Правило - информация (fb)](https://public.tableau.com/profile/publish/Rule-Information-SocialDatafb/Dashboard2#!/publish-confirm)     
-  [Правило - информация (mm)](https://public.tableau.com/profile/publish/Rule-Information-SocialDatamm/Dashboard1#!/publish-confirm)    
-  [Правило - информация (ok)](https://public.tableau.com/profile/publish/Rule-Information-SocialDataok/Dashboard3#!/publish-confirm)    
-  [Правило - информация (vk)](https://public.tableau.com/profile/publish/Rule-Information-SocialDatavk/Dashboard4#!/publish-confirm)    
Данные дэшборды показывают получившиеся правила и информацию о них (support, confidence, lift, содержание страны (соц. данные) в правиле, содержание акционного пака).

**Порядок правил не учитывается!** 

## *Выводы по 2-му уровню:*


Во всех соц. сетях получили достаточно много промежутков при транзакциях более, чем 100. Однако верхняя граница транзакций у социальных сетей fb и mm не превышает 400 (fb - 380, mm - 308). У vk и ok эта граница выше (700 и 1067 соответственно).

Акционные паки: 

При выделении правил, учитывая реккомендации (транзакций более 100, остальное на минимальном значении) получаются такие результаты:

1. По социальной сети **fb** выходит 0 правил (что также является важным результатом, с учетом того, что в первом уровне рассматривались все социальные сети, а здесь только **fb**, т.е. гораздо меньше шанс потерять правило из-за огромного количестива транзакций).

2. По остальным социальным сетям выходят некоторые правила, но абсолютно все они такого вида: пак => страна в котором он куплен (абсолютно тривиальные и не представляющие важности правила).

**Вывод:** учитывая то, что в предыдущем уровне не было найдено правил с акционными паками (при рекомендуемых параметрах), то добавление страны, ожидаемо, не должно было изменить результата. Однако ограниченность одной соц. сетью позволяет сделать новые, но в то же время уже известные нам выводы: либо мы испытываем **недостаток данных** , либо мы наблюдаем **абсолютное отсутствие влияния акционных паков на покупки**.

# *LEVEL 3:*

### Важно отметить:

Стратегии было решено искать также, используя Априори. Для каждого пользователя проверялось выполнялось ли для него правило (порядок учитывается) в конкретном периоде (периоды, как в (3) дэшборде 1-го уровня). Таким образом транзакция состояла из различных правил, найденных в 1-м уровне.

Запускаем Априори (без определения порядка) на транзакциях по правилам (ищем стратегии):

- **стратегии без определения порядка!**
- **правила с определением порядка!**

In [14]:
t_d_1={}
with open('file_for_transactions.csv', 'w') as csf:
    for key, value in rules_for_apriori.items():
        st=""
        for i in range(0,len(value)):
            st=st+value[i]+','
        csf.write(st+'\n')


inFile = m.dataFromFile("file_for_transactions.csv")             
items, rules = m.runApriori_lvl3(inFile, 0.02, 0.5)  

with open('for_test_res_lvl3.csv', 'w') as rs:
    id_count=0
    id_count, t_d_1=m.printResults(items, rules, rs, id_count, t_d_1, 0, 1, 2, action_pack)
                    
                    

Создаем файлы (правила-информация и инициализация правил) для Tableau:

In [15]:
with open('g_t_strategy.csv', 'w') as csvfile_1:
    fieldnames=["ID","Rule","Confidence","Lift","Support"]
    writer = csv.DictWriter(csvfile_1, fieldnames=fieldnames,delimiter='|')
    writer.writeheader()
    for key,value in t_d_1.items():
        #print value[4],
        writer.writerow({fieldnames[0]: key, fieldnames[1]: value[0], fieldnames[2]: value[1], fieldnames[3]: value[2],fieldnames[4]: value[3]})

In [16]:
with open('definition.csv', 'w') as csvfile_1:
    fieldnames=["Rule", "Rule's definition","Date"]
    writer = csv.DictWriter(csvfile_1, fieldnames=fieldnames,delimiter='|')
    writer.writeheader()
    for key,value in rules_definition.items():
        writer.writerow({fieldnames[0]: value[0], fieldnames[1]: key,fieldnames[2]: value[1]})

## *Результаты по 3-му уровню*:

-  [Стретегии - инициализация правил](https://public.tableau.com/profile/publish/Strat-Def/Dashboard1#!/publish-confirm)     
Данный дэшборд показывает получившиеся стратегии, информацию о них (support, confidence, lift) и иницилизацию правил в стратегиях. Транзакции правил строились по информации, которую можно наблюдать в  "[правило - информация (3)](https://public.tableau.com/profile/publish/Rule-Informationconsideringsequenceandperiods/Dashboard1#!/publish-confirm)".     

**Порядок правил в стратегиях не учитывается!** Можно проследить по дэшборду с инициализациями правил.

## *Выводы по 3-му уровню:*

Получившиеся стретегии, после изучения появления паков в игре, оказались достаточно тривиальными (пример: пак  *Х* можно купить только (!) после того, как купишь пак *У*). Однако этот результат показал, что стратегии по Априори выявить можно, а отсутствие информации об акционных паках **можно трактовать** либо **как недостаток данных**, либо как то, что **акционные паки не оказывают существенного влияния на покупки**.